## Generando una población una realizar una prueba-error de los análisis de nuestra encuesta
Para ello utilizaremos el *método de composición* para la generación de *variables discretas aleatorias*.

### Objetivo
    Generar una población tal que cada individuo tenga respuestas coherentes dado su nivel de humor.

### Previo
Para nuestra encuesta tenemos 60 preguntas repartidas en 3 categorías:
* Alegría
* Seriedad
* Mal Humor

A su vez cada categoría se divide en subcategorías, el individuo deberá responder de acuerdo a que tan de acerdo esta con la pregunta: completamente en desacuerdo, moderadamente en desacuerdo, moderadamente de acuerdo y completamente de acuerdo. Cada respuesta tendrá un valor numérico de 1 a 4 respectivamente.

Por lo tanto lo que se genera son valores discretos: 1, 2, 3, 4. Para ellos se necesita que por individuo las respuestas sean coherentes de modo que sea condicionado por su nivel de humor generado con una función de distribución normal sh~U(0, 1)

Utlizando sh vamos a estimar para cada categoría que respuesta debería dar el individuo de tal forma que si sh<30 es más probable que las preguntas correspondinetes a medir el mal humor esten respondidas con de acuerdo, las de seriedad tambien en de acuerdo ademas de moderadamente en desacuerdo, y las de alegria en desacuerdo. por lo tanto utilizando el método de composición hacemos una separación por caso y devolvemos los valores por subcategoría, teniendo previamente un diccionario que relaciona categoría con pregunta podemos dado el valor esperado de cada categoría generar con una distribución normal el valor que responderá en esa pregunta teniendo el promedio como el valor generado en esa subcategoría y una variancia pequeña.

### Código
Primero creemos el diccionario con la relación pregunta: categoría

In [1]:
#relación pregunta subcategoría de tal forma que las subcategorías corresponden a un rango
# Alegría de 0 a 4, seriedad de 5 a 9 y mal humor de 10 a 14
rel = {
    4:0, 19:0, 32:0, 46:0, 50:0,
    9:1, 22:1, 30:1,
    2:2, 14:2, 35:2, 53:2,
    16:3, 26:3, 38:3, 44:3,
    25:4, 41:4, 57:4, 59:4,
    5:5, 7:5, 15:5, 33:5, 
    18:6, 28:6, 39:6, 49:6, 
    12:7, 23:7, 47:7, 60:7,
    3:8, 20:8, 42:8, 52:8, 
    10:9, 36:9, 55:9, 58:9, 
    1:10, 31:10, 34:10, 37:10, 51:10, 
    8:11, 13:11, 29:11, 40:11, 54:11, 56:11, 
    21:12, 48:12, 
    6:13, 11:13, 17:13, 27:13, 43:13, 45:13, 
    24:14
}

Luego creamos la función que nos genera la respuesta de un individuo, devuelve una lista con las 60 respuestas

In [2]:
import numpy as np

def response(n):
    # Generando el sentido del humor de un individuo con una distribución uniforme de 0 a 1
    sh = np.random.uniform(0,1,1)[0]

    # Listas de categorias al -> alegría, sr -> seriedad, mh -> mal humor
    # Hacemos una división por caso dependiendo del sh dado
    if sh < 0.3:
        # alegria -> 1,2; seriedad -> 2,3,4; mal humor -> 3,4
        al = np.random.normal(1, 0.5, 5)
        sr = np.random.normal(3, 0.8, 5)
        mh = np.random.normal(4, 0.5, 5)    
    elif sh < 0.5:
        # alegria -> 1,2,3; seriedad -> 2,3,4; mal humor -> 2,3,4
        al = np.random.normal(2, 0.8, 5)
        sr = np.random.normal(3, 0.8, 5)
        mh = np.random.normal(3, 0.8, 5)
    elif sh < 0.7:
        # alegria -> 2,3; seriedad -> 2,3; mal humor -> 1,2,3
        al = np.random.normal(3, 0.5, 5)
        sr = np.random.normal(3, 0.5, 5)
        mh = np.random.normal(2, 8, 5)
    else:
        # alegria -> 3,4; seriedad -> 1,2,3; mal humor -> 1,2
        al = np.random.normal(4, 0.5, 5)
        sr = np.random.normal(2, 0.8, 5)
        mh = np.random.normal(1, 0.5, 5)

    # De esta forma hemos asugnado para cada subcategoría un valor, no necesariamente entre 1 y 4, tampoco enteros
    # pero si que tienden a los valores que esperamos dado el sentido del humor generado aleatoriamente
    # A continuación generaremos las respuestas de las siguientes preguntas usando la distribución normal 
    # con media en el valor generado y una varianza de a penas 0.5 para de esta forma genrar valores cercanos al que necesitmos
    cat = np.concatenate([al,sr,mh])

    r = [n]
    for i in range(60):
        p = np.random.normal(cat[rel[i+1]], 0.2, 1)[0]
        p = round(p)
        if p<1:
            p=1
        if p >4:
            p=4
        r.append(p)

    return r

A continuación la función que genera una población de tamaño n

In [3]:
def genPop(n):
    #A continuación se procede a crear una población de tamaño n
    poblacion = []
    for i in range(n): 
        poblacion.append(response(i))
    
    return poblacion

Teniendo nuestra función de generación de una población procedemos a leer y llenar la hoja de cálculo con una población generada aleatoriamente y en la primera fila las preguntas de la encuesta

In [4]:
pop = genPop(120)

import pandas as pd

df = pd.DataFrame()
df = df.append(pop)

df.columns = ["Nombre", "Las personas se dan cuenta cuando algo me molesta","Mi forma de vida puede describirse como positiva y libre de preocupaciones","Yo muy rara vez actuo sin una debida razon","Yo soy una persona contenta","La mayoria de mis amigos son mas bien serios y reflexivos","Algunas circunstancias molestas son capaces de echar a perder mi estado de animo un buen rato","Prefiero conversar de asuntos que sean importantes y profundos","A veces tengo la sensacion de un vacio interior","Me pueden hacer reir facilmente","No me gusta que la gente exagere cuando habla conmigo","Comparado con los otros yo puedo ser bastante grunnon(a)","Yo planeo mis acciones y tomo decisiones solo en la medida que me son utiles en el largo plazo","A menudo me siento desalentado (a)","Yo puedo relajarme facilmente y disfrutar el momento","Yo soy una persona seria","La vida diaria frecuentemente me da la ocasion de reir","Frecuentemente pienso: por Dios, no me molesten hoy dia","En mi vida me gusta tener todo ordenado","Yo soy alegre por naturaleza","Cuando veo la television prefiero los reportajes a los programas superficiales","Cuando estoy alterado(a) ni aun las cosas mas divertidas son capaces de alegrarme","Yo sonrio frecuentemente","En todas las cosas que hago siempre considero los posibles efectos y comparo los pro y los contra cuidadosamente","Cuando mis amigos tratan de alegrarme con bromas o tonteras, a veces me pongo mas malhumorado (a) y grunnon(a)","La risa tiene un efecto contagioso en mi","Frecuentemente encuentro que las pequennas cosas de la vida son comicas y entretenidas","Hay muchos dias en los que pienso: Hoy me levante con el pie izquierdo","En la mayoria de las situaciones desde un principio me fijo en los aspectos serios","A veces me siento triste sin razon","Me gusta reir y lo hago frecuentemente","Mi animo a menudo no es de los mejores","Soy una persona feliz","Cuando estoy en contacto con otros, con frecuencia encuentro que he pensado las cosas mas profundamente que ellos","Aunque no haya ninguna razon, con frecuencia me siento mal humorado/a","Muchas de las dificultades del diario vivir tienen su lado positivo","Cuando converso siempre evito las exageraciones, adornos y ambig�uedades, o sea todo lo que no contribuya al significado claro de lo que digo","Con frecuencia estoy mal de animo","Me siento muy a gusto compartiendo con gente alegre","Mi vida esta compuesta principalmente de cosas importantes","Algunas veces ando alterado(a) durante largo tiempo","El buen estado de animo de los otros tiene un efecto contagioso en mi","Yo no entiendo como los otros pueden gastar su tiempo en cosas sin sentido","Estoy frecuentemente irritable","Con frecuencia encuentro que los pequennos contratiempos de la vida diaria me divierten, incluso si me pasan a mi","Mis conocidos con frecuencia me irritan","Siempre estoy de buen animo, aunque no tenga una razon especifica","Tiendo a hacer planes para el futuro y con metas de largo plazo","Generalmente me siento tan amargado que nada me hace reir","Aun las cosas mas triviales las trato con seriedad y responsabilidad","A menudo estoy de un animo alegre","Si estoy de mal animo no puedo estar con personas que estan contentas","Trato de utilizar mi tiempo libre haciendo en lo posible cosas utiles","La experiencia me ha demostrado que el proverbio: la risa es la mejor medicina, es realmente cierta","A ratos soy una persona triste","Prefiero las personas que se comunican en forma reflexiva y objetiva","Con frecuencia me siento tan cansado (a) que no puedo motivarme para hacer nada","Me gusta bromear con otros","Cuando me comunico con otra gente, siempre trato de tener un intercambio de ideas objetivo y sobrio","Es facil para mi repartir a otros buena onda","Uno de mis lemas es: trabaja primero, juega despues"]

df.to_csv("STCI.csv")


In [5]:
subcat = {
    0:"Prevalencia de animo alegre.",
    1:"Bajo umbral para la risa y sonrisa.",
    2:"Vision positiva de las circunstancias adversas de la vida.",
    3:"Amplio rango de desencadenadores de alegria, risa y sonrisa",
    4:"Estilo de interaccion alegre",
    5:"Prevalencia de estados serios.",
    6:"Percepcion de que los eventos de la vida diaria son serios e importantes.",
    7:"Vivir en torno a metas.",
    8:"Preferir actividades concretas y racionales",
    9:"Estilo de comunicacion sobrio.",
    10:"Prevalencia de mal humor",
    11:"Prevalencia de tristeza",
    12:"Tristeza en situaciones alegres.",
    13:"Sentimientos de irritabilidad.",
    14:"Irritabilidad en situaciones alegres.",
}
preg = ["Las personas se dan cuenta cuando algo me molesta","Mi forma de vida puede describirse como positiva y libre de preocupaciones","Yo muy rara vez actuo sin una debida razon","Yo soy una persona contenta","La mayoria de mis amigos son mas bien serios y reflexivos","Algunas circunstancias molestas son capaces de echar a perder mi estado de animo un buen rato","Prefiero conversar de asuntos que sean importantes y profundos","A veces tengo la sensacion de un vacio interior","Me pueden hacer reir facilmente","No me gusta que la gente exagere cuando habla conmigo","Comparado con los otros yo puedo ser bastante grunnon(a)","Yo planeo mis acciones y tomo decisiones solo en la medida que me son utiles en el largo plazo","A menudo me siento desalentado (a)","Yo puedo relajarme facilmente y disfrutar el momento","Yo soy una persona seria","La vida diaria frecuentemente me da la ocasion de reir","Frecuentemente pienso: por Dios, no me molesten hoy dia","En mi vida me gusta tener todo ordenado","Yo soy alegre por naturaleza","Cuando veo la television prefiero los reportajes a los programas superficiales","Cuando estoy alterado(a) ni aun las cosas mas divertidas son capaces de alegrarme","Yo sonrio frecuentemente","En todas las cosas que hago siempre considero los posibles efectos y comparo los pro y los contra cuidadosamente","Cuando mis amigos tratan de alegrarme con bromas o tonteras, a veces me pongo mas malhumorado (a) y grunnon(a)","La risa tiene un efecto contagioso en mi","Frecuentemente encuentro que las pequennas cosas de la vida son comicas y entretenidas","Hay muchos dias en los que pienso: Hoy me levante con el pie izquierdo","En la mayoria de las situaciones desde un principio me fijo en los aspectos serios","A veces me siento triste sin razon","Me gusta reir y lo hago frecuentemente","Mi animo a menudo no es de los mejores","Soy una persona feliz","Cuando estoy en contacto con otros, con frecuencia encuentro que he pensado las cosas mas profundamente que ellos","Aunque no haya ninguna razon, con frecuencia me siento mal humorado/a","Muchas de las dificultades del diario vivir tienen su lado positivo","Cuando converso siempre evito las exageraciones, adornos y ambig�uedades, o sea todo lo que no contribuya al significado claro de lo que digo","Con frecuencia estoy mal de animo","Me siento muy a gusto compartiendo con gente alegre","Mi vida esta compuesta principalmente de cosas importantes","Algunas veces ando alterado(a) durante largo tiempo","El buen estado de animo de los otros tiene un efecto contagioso en mi","Yo no entiendo como los otros pueden gastar su tiempo en cosas sin sentido","Estoy frecuentemente irritable","Con frecuencia encuentro que los pequennos contratiempos de la vida diaria me divierten, incluso si me pasan a mi","Mis conocidos con frecuencia me irritan","Siempre estoy de buen animo, aunque no tenga una razon especifica","Tiendo a hacer planes para el futuro y con metas de largo plazo","Generalmente me siento tan amargado que nada me hace reir","Aun las cosas mas triviales las trato con seriedad y responsabilidad","A menudo estoy de un animo alegre","Si estoy de mal animo no puedo estar con personas que estan contentas","Trato de utilizar mi tiempo libre haciendo en lo posible cosas utiles","La experiencia me ha demostrado que el proverbio: la risa es la mejor medicina, es realmente cierta","A ratos soy una persona triste","Prefiero las personas que se comunican en forma reflexiva y objetiva","Con frecuencia me siento tan cansado (a) que no puedo motivarme para hacer nada","Me gusta bromear con otros","Cuando me comunico con otra gente, siempre trato de tener un intercambio de ideas objetivo y sobrio","Es facil para mi repartir a otros buena onda","Uno de mis lemas es: trabaja primero, juega despues"]
sub =[]
for i in range(60):
    app = [i+1, preg[i], rel[i+1], subcat[rel[i+1]]]
    if rel[i+1]<5:
        app.append("Alegria")
    elif rel[i+1]<10:
        app.append("Seriedad")
    else:
        app.append("Mal humor")
    sub.append(app)

dfRel = pd.DataFrame()

dfRel = dfRel.append(sub)

dfRel.columns = ["NumeroPregunta", "Pregunta", "IndiceSubCategoria", "SubCategoria", "Categoria"]
dfRel.set_index("NumeroPregunta", inplace=True)

dfRel.to_csv("PreguntaCategoria.csv")